In [1]:
import rclpy as rp
import jupyros.ros2 as jr2
import jupyros.ros2.turtle_sim as turtle
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from time import time
import os
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sidecar import Sidecar
from time import time, sleep
import math



In [2]:
# Initialize ROS communications for a given context
if(rp.ok() == False):
    rp.init()
else:
    print("rclpy already initiated")

In [3]:
superturtle = rp.create_node("superturtle")
moveNode = rp.create_node("moveNode")

In [4]:
turtlesim = turtle.TurtleSim(background_color="#0000FF")

turtle1 has spawned.


In [5]:
display(turtlesim.canvas)

MultiCanvas(height=1600, layout=Layout(width='100%'), width=1600)

**TIP:** When using JupyterLab, you can right-click on the canvas and select *Create New View from Output*

In [6]:
poses = {}

for name in turtlesim.turtles.keys():
    poses[name] = turtlesim.turtles[name].pose
    
print(poses["turtle1"])

{'x': 800, 'y': 800, 'theta': 0}


In [7]:
topic_name = '/keyboard_stream'
def move_turtles(msg):
    scale = 1
    name = "turtle1"

    poses[name] = {"x": turtlesim.turtles[name].pose["x"] + msg.linear.x*math.cos(turtlesim.turtles[name].pose["theta"])/scale,
                   "y": turtlesim.turtles[name].pose["y"] - msg.linear.x*math.sin(turtlesim.turtles[name].pose["theta"] )/scale,
                   "theta": turtlesim.turtles[name].pose["theta"] + msg.angular.z/ 180 * math.pi}
    
  
    turtlesim.move_turtles(new_poses=poses)




def cb(msg):
    print(msg)
    move_turtles(msg)


turtle_control = jr2.Subscriber(moveNode, Pose, topic_name, cb)
turtle_control.display()

In [8]:
%%thread_cell2
run = True
i = 0
pub = jr2.Publisher(moveNode, Pose, topic_name)

while run:
    msg = Pose()
    msg.x = (i/1800*5)*math.sin(i / 180 * math.pi) + 11.08 / 2
    msg.y = (i/1800*5)*math.cos(i / 180 * math.pi) + 11.08 / 2
    msg.theta = - i / 180 * math.pi
    pub.send_msg(msg)
    i += 1
print("Done")

Output(layout=Layout(border='1px solid gray'))